<a href="https://colab.research.google.com/github/seunghwan1228/TF-Torch/blob/main/tensorflow_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import numpy as np
import os
import matplotlib.pyplot as plt
import random

import tensorflow as tf


from tqdm.notebook import tqdm

import time

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [20]:
@tf.function
def scale(x, y):
  x = tf.cast(x, tf.float32)
  x = tf.expand_dims(x, -1)
  x = x/255.
  return (x, y)


def data_loader(x, y):
  ds = tf.data.Dataset.from_tensor_slices((x,y))
  ds = ds.map(scale)
  ds = ds.batch(512)
  ds = ds.prefetch(tf.data.AUTOTUNE)
  return ds

In [21]:
train_loader = data_loader(x_train, y_train)
valid_loader = data_loader(x_test, y_test)

In [22]:
train_loader.cardinality()

<tf.Tensor: shape=(), dtype=int64, numpy=118>

In [23]:
tmp_img, tmp_label = next(iter(train_loader))

In [25]:
tmp_img.shape, tf.reduce_mean(tmp_img), tf.math.reduce_std(tmp_img)

(TensorShape([512, 28, 28, 1]),
 <tf.Tensor: shape=(), dtype=float32, numpy=0.12575655>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.30235657>)

In [42]:
class MnistLinearModel(tf.keras.Model):
  def __init__(self, num_units, num_layers, output_classes):
    super(MnistLinearModel, self).__init__()

    self.input_layer = tf.keras.layers.Dense(num_units, activation='relu')
    self.hiddens = [tf.keras.layers.Dense(num_units, activation='relu') for _ in range(num_layers)]
    self.output_layer = tf.keras.layers.Dense(output_classes)

  def call(self, inputs):
    x = tf.keras.layers.Flatten()(inputs)
    x = self.input_layer(x)
    for l in self.hiddens:
      x = l(x)
    return self.output_layer(x)

In [43]:
tmp_model = MnistLinearModel(32, 5, 10)

In [44]:
tmp_model.build(input_shape=(28,28,1))

In [45]:
tmp_model.summary()

Model: "mnist_linear_model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             multiple                  928       
_________________________________________________________________
dense_28 (Dense)             multiple                  1056      
_________________________________________________________________
dense_29 (Dense)             multiple                  1056      
_________________________________________________________________
dense_30 (Dense)             multiple                  1056      
_________________________________________________________________
dense_31 (Dense)             multiple                  1056      
_________________________________________________________________
dense_32 (Dense)             multiple                  1056      
_________________________________________________________________
dense_33 (Dense)             multiple         

In [67]:
def train(model, train_data, valid_data, loss_module, optimizer, epochs=100):
  __start = time.time()
  train_acc = tf.keras.metrics.SparseCategoricalAccuracy()
  train_loss = tf.keras.metrics.Mean(name='Train-Loss')
  valid_acc = tf.keras.metrics.SparseCategoricalAccuracy()
  valid_loss = tf.keras.metrics.Mean(name='Valid-Loss')

  for epoch in tqdm(range(1, epochs+1)):
    train_acc.reset_states()
    valid_acc.reset_states()
    
    train_loss.reset_states()
    valid_loss.reset_states()

    # Train
    for img, label in train_data:
      with tf.GradientTape() as tape:
        pred = model(img)
        loss = loss_module(label, pred)
      # grad = tape.gradient(loss, model.trainable_variables)
      # optimizer.apply_gradients(zip(grad, model.trainable_variables))
      optimizer.minimize(loss, model.trainable_variables, tape=tape)
      train_loss(loss)
      train_acc(label, pred)
    
    if epoch % 10 == 0:
      print(f'epoch {epoch}\tTrain: ',train_loss.result().numpy(), train_acc.result().numpy())
    
    for v_img, v_label in valid_data:
      v_pred = model(v_img)
      v_loss = loss_module(v_label, v_pred)

      valid_acc(v_label, v_pred)
      valid_loss(v_loss)
    
    if epoch % 10 == 0:
      print(f'epoch {epoch}\tValid: ',valid_loss.result().numpy(), valid_acc.result().numpy())
  print(f'Complete: {time.time() - __start}')

In [68]:
# GPU K80
model = MnistLinearModel(32, 5, 10)
loss_module = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Nadam()


train(model, train_loader, valid_loader, loss_module, optimizer)

  0%|          | 0/100 [00:00<?, ?it/s]

epoch 10	Train:  0.11804655 0.9650667
epoch 10	Valid:  0.1625019 0.9544
epoch 20	Train:  0.06685985 0.98083335
epoch 20	Valid:  0.14951816 0.9598
epoch 30	Train:  0.038822394 0.98965
epoch 30	Valid:  0.16161345 0.9615
epoch 40	Train:  0.024791006 0.99303335
epoch 40	Valid:  0.1977149 0.9609
epoch 50	Train:  0.022503546 0.99325
epoch 50	Valid:  0.23402147 0.9598
epoch 60	Train:  0.010090233 0.99695
epoch 60	Valid:  0.26583478 0.9582
epoch 70	Train:  0.009589172 0.99691665
epoch 70	Valid:  0.3103107 0.9583
epoch 80	Train:  0.008987488 0.99698335
epoch 80	Valid:  0.3208811 0.961
epoch 90	Train:  0.00970739 0.99663335
epoch 90	Valid:  0.380569 0.9551
epoch 100	Train:  0.0053459005 0.9982
epoch 100	Valid:  0.3472313 0.9615
Complete: 981.1518929004669
